In [14]:
import pandas as pd

with open('../data/SaleDataset.csv', 'r') as f:
    data = pd.read_csv(f)

In [15]:
import pandas as pd
import numpy as np

def Convert_Ordinal(data:pd.DataFrame) -> pd.DataFrame:

    out = data.copy()

    #MSSubClass should be categorical
    mapping = {
        20:'1-STORY 1946 & NEWER ALL STYLES',
        30:'1-STORY 1945 & OLDER',
        40:'1-STORY W/FINISHED ATTIC ALL AGES',
        45:'1-1/2 STORY - UNFINISHED ALL AGES',
        50:'1-1/2 STORY FINISHED ALL AGES',
        60:'2-STORY 1946 & NEWER',
        70:'2-STORY 1945 & OLDER',
        75:'2-1/2 STORY ALL AGES',
        80:'SPLIT OR MULTI-LEVEL',
        85:'SPLIT FOYER',
        90:'DUPLEX - ALL STYLES AND AGES',
        120:'1-STORY PUD (Planned Unit Development) - 1946 & NEWER',
        150:'1-1/2 STORY PUD - ALL AGES',
        160:'2-STORY PUD - 1946 & NEWER',
        180:'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER',
        190:'2 FAMILY CONVERSION - ALL STYLES AND AGES'
    }
    out['MSSubClass'] = out['MSSubClass'].replace(mapping)

    #NA is not value, but legit NA; Doesn't matter with later .fillna(0)
    #Quality and Condition
    mapping = {
        'Ex':2,
        'Gd':1,
        'TA':0,
        'Fa':-1,
        'Po':-2,
        'NA':-10,
    }
    col_apply = [
        'ExterQual', 'ExterCond',
        'BsmtQual', 'BsmtCond',
        'HeatingQC',
        'KitchenQual',
        'GarageQual', 'GarageCond',
        'FireplaceQu'
    ]
    for col in col_apply: 
        out[col] = out[col].replace(mapping)
        out[col] = out[col].fillna(-5)

    #Basement Exposure
    mapping = {
        'Gd':3,
        'Av':2,
        'Mn':1,
        'No':0
    }
    out['BsmtExposure'] = out['BsmtExposure'].replace(mapping)

    #Basement Finish
    mapping = {
        'GLQ':5,
        'ALQ':4,
        'BLQ':3,
        'Rec':2,
        'LwQ':1,
        'Unf':0,
    }
    out['BsmtFinType2'] = out['BsmtFinType2'].replace(mapping)

    #Deductions
    mapping = {
        'Typ':0,
        'Min1':1,
        'Min2':2,
        'Mod':3,
        'Maj1':4,
        'Maj2':5,
        'Sev':6,
        'Sal':7
    }
    out['Functional'] = out['Functional'].replace(mapping)

    #Garage Finsish
    mapping = {
        'Fin':1,
        'RFn':0.5,
        'Unf':0
    }
    out['GarageFinish'] = out['GarageFinish'].replace(mapping)

    #PavedDrive
    mapping = {
        'Y':1,
        'P':0.5,
        'N':0
    }
    out['PavedDrive'] = out['PavedDrive'].replace(mapping)   

    #Fence Quality
    mapping = {
        'GdPrv':4,
        'MnPrv':3,
        'GdWo':2,
        'MnWw':1,
        'NA':0
    } 
    out['Fence'] = out['Fence'].replace(mapping)  
    out['Fence'] = out['Fence'].fillna(0)

    return out

def Noncategorical_Dummies(data:pd.DataFrame) -> pd.DataFrame:

    out = data.copy()

    out['CentralAir'] = np.where(out['CentralAir'] == 'N', 0, 1)

    out['Remodeled'] = np.where(out['YearRemodAdd'] == out['YearBuilt'], 0, 1)
    
    out['MasonryVeneer'] = np.where(out['MasVnrType'].isna(), 0, 1)

    out['AlleyAccess'] = np.where(out['Alley'].isna(), 0, 1)

    out['Basement'] = np.where(out['BsmtExposure'].isna(), 0, 1)

    out['Garage'] = np.where(out['GarageType'].isna(), 0, 1)

    return out

def Summarise_Variables(data:pd.DataFrame) -> pd.DataFrame:

    out = data.copy()

    out['BsmtBathrooms'] = out['BsmtFullBath'] + 0.5*out['BsmtHalfBath']
    out = out.drop(columns=['BsmtFullBath', 'BsmtHalfBath'])

    out['Bathrooms'] = out['FullBath'] + 0.5*out['HalfBath']
    out = out.drop(columns=['FullBath', 'HalfBath'])

    out['Age'] = out['YearBuilt'].max() - out['YearBuilt']
    out['Remodeled_Age'] = out['YearRemodAdd'].max() - out['YearRemodAdd']

    return out


data = Noncategorical_Dummies(data)
data = Convert_Ordinal(data)
data = Summarise_Variables(data)

#Utilities different for only 1 property
#Masonry, Alley missing for most, converted to bool
#Above grade living area is sum of 1st and 2nd Floor IN MOST CASES!!!
#MiscFeature rare, kinda included in MiscValue
data = data.drop(columns=['Utilities', 'MasVnrArea', 'MasVnrType', 'Alley', 'GrLivArea', 'MiscFeature', 'Id'])
data = data.fillna(0)

C:\Users\karli\AppData\Local\Temp\ipykernel_63848\2823184786.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out[col] = out[col].replace(mapping)
C:\Users\karli\AppData\Local\Temp\ipykernel_63848\2823184786.py:58: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out['BsmtExposure'] = out['BsmtExposure'].replace(mapping)
C:\Users\karli\AppData\Local\Temp\ipykernel_63848\2823184786.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly c

In [16]:
import pandas as pd

with open('../data/DatasetCleaned.csv', 'w') as f:
    data.to_csv(f)